In [3]:
%%writefile src/Multiply7Event.sol
pragma solidity ^0.5.10;
contract Multiply7Event {
    event Print(address _addr, uint256 timestamp, uint res);
    //function multiply(uint input) public returns(uint) {
    function multiply(uint input) public {
        uint res = input * 7;
        emit Print(msg.sender, now, res);
        //return res;
    }
}

Overwriting src/Multiply7Event.sol


In [4]:
!solc --gas --abi --bin src/Multiply7Event.sol


======= src/Multiply7Event.sol:Multiply7Event =======
Gas estimation:
construction:
   105 + 53600 = 53705
external:
   multiply(uint256):	1862
Binary: 
6080604052348015600f57600080fd5b5061010c8061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c6888fa114602d575b600080fd5b605660048036036020811015604157600080fd5b81019080803590602001909291905050506058565b005b60006007820290507f91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205334283604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001838152602001828152602001935050505060405180910390a1505056fea265627a7a72305820617b9abfc7fae820c6ca8a5993a0554c413f36b43d73915d2250dbd5f00995c764736f6c634300050a0032
Contract JSON ABI 
[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"_addr","type"

In [6]:
%%writefile src/Multiply7EventDeploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray=[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"_addr","type":"address"},{"indexed":false,"name":"timestamp","type":"uint256"},{"indexed":false,"name":"res","type":"uint256"}],"name":"Print","type":"event"}];
var _bin="0x" + "6080604052348015600f57600080fd5b5061010c8061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c6888fa114602d575b600080fd5b605660048036036020811015604157600080fd5b81019080803590602001909291905050506058565b005b60006007820290507f91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205334283604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001838152602001828152602001935050505060405180910390a1505056fea265627a7a72305820617b9abfc7fae820c6ca8a5993a0554c413f36b43d73915d2250dbd5f00995c764736f6c634300050a0032";
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance) {
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/Multiply7EventDeploy.js


In [7]:
!node src/Multiply7EventDeploy.js

Deploying the contract from 0x0c6128daD2232C486Ed51B64D38E50910039BDBc
hash: 0x5ec4c94c0343710060b043fb8327a92e4baeed971102f558f14cf858559e4b6a
---> The contract deployed to: 0x1be2de72761B581b825e874cdAAdd3c56000b3F7


In [8]:
%%writefile src/Multiply7EventUse.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
}
var _abiArray=[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"_addr","type":"address"},{"indexed":false,"name":"timestamp","type":"uint256"},{"indexed":false,"name":"res","type":"uint256"}],"name":"Print","type":"event"}];
var m7 = new web3.eth.Contract(_abiArray, '0x1be2de72761B581b825e874cdAAdd3c56000b3F7');
var event = m7.events.Print({fromBlock: 0}, function (error, result) {
    if (!error) {
        //console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
        console.log("Event fired: " + JSON.stringify(result.returnValues));
        //process.exit(1);
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    //const value = m7.methods.multiply(8).call();
    const value = m7.methods.multiply(8)
        .send({from: accounts[0]})
        //.then(function(value) {console.log("---> MyFunction called " + JSON.stringify(value) +
        //                               '\n---> '+ JSON.stringify(value.events.Print.returnValues));});
    console.log("---> MyFunction called " + JSON.stringify(value));
        //+ '\n---> '+ JSON.stringify(value.events.Print.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/Multiply7EventUse.js


In [9]:
!node src/Multiply7EventUse.js

Account: 0x0c6128daD2232C486Ed51B64D38E50910039BDBc
Balance before: 99999888655000000000
---> MyFunction called {"_events":{}}
Balance after: 99999888655000000000
Balance diff: 0
Event fired: {"0":"0x0c6128daD2232C486Ed51B64D38E50910039BDBc","1":"1591930833","2":"56","_addr":"0x0c6128daD2232C486Ed51B64D38E50910039BDBc","timestamp":"1591930833","res":"56"}
